In [1]:
import requests
from tqdm import tqdm_notebook
import time
import pickle

### Get token

In [2]:
url_token =  'https://id.twitch.tv/oauth2/authorize?response_type=token&client_id=jh0fr7g4isys0t2fmviygsvk7ez3l9&redirect_uri=http://localhost&scope=channel_read'

### Get User List

In [3]:
df_streamList = pd.read_csv("data/streamList.csv")
li_channelID = df_streamList.user_id.tolist()

In [4]:
df_streamList.tail()

,user_name,user_id,Nickname,Link
245,soming1907,138029395,소밍,/c/138029395-soming1907
246,munchikinhwa,102522459,먼치킨화,/c/102522459-munchikinhwa
247,innovation_s2,48460548,이노베이션,/c/48460548-innovation_s2
248,myosonge,137962705,묘송이,/c/137962705-myosonge
249,gj7dudwo,137432936,A후라,/c/137432936-gj7dudwo


In [5]:
header = {
    "Accept" : "application/vnd.twitchtv.v5+json",
    "Client-ID" : "jh0fr7g4isys0t2fmviygsvk7ez3l9",
}

In [6]:
# df_user_list = pd.DataFrame(columns = ["id", "name"])

250명의 스트림머의 follower 1000명씩 가져오기

In [7]:
# for channel in tqdm_notebook(li_channelID):
#     for page in range(10):
#         url_follow = "https://api.twitch.tv/kraken/channels/" + str(channel) + "/follows?limit=100&offset=" + str(page)
#         response_follow = requests.get(url_follow, headers = header)

#         res_json = response_follow.json()

#         for idx in range(len(res_json["follows"])):
#             user_id = res_json["follows"][idx]["user"]["_id"]
#             user_name = res_json["follows"][idx]["user"]["name"]
#             df_user_list.loc[len(df_user_list)] = {"id" : user_id, "name" : user_name}

In [8]:
# df_list.to_csv("data/user_list_all.csv", index = False)

In [9]:
df_user_list = pd.read_csv("data/user_list_all.csv")

In [10]:
df_user_list.shape

(250000, 2)

### unique user list 뽑기

In [11]:
# df_unique_user = df_user_list.drop_duplicates(subset=['id']).reset_index(drop=True)

In [12]:
# df_unique_user.to_csv("data/unique_user_list.csv", index = False)

In [13]:
df_unique_user = pd.read_csv("data/unique_user_list.csv")

### Get follower list by the user

In [14]:
li_unique_user = df_unique_user.id.tolist()
len(li_unique_user)

21815

In [194]:
df_user_following = pd.DataFrame(columns = ["id", "following_list"])

In [58]:
for user_id in tqdm_notebook(c):
    url_followlist = "https://api.twitch.tv/helix/users/follows?from_id=" + str(user_id) + "&first=100"
    response_followlist = requests.get(url_followlist, headers = header)
    res_json = response_followlist.json()
    
    try :
        # follower 갯수 받아오기
        num_following = res_json["total"]
        # follower 리스트 받아오기
        li_following = []
        bIter = True
        while(bIter == True):
            # set iterate number
            if(num_following > 100):
                num_iter = 100
                num_following -= 100
                cursor = res_json["pagination"]["cursor"]
            else: 
                num_iter = num_following
                bIter = False

            # get follower list
            for idx_following in range(len(res_json['data'])):
                li_following.append(res_json['data'][idx_following]["to_id"])
            
           # url 다시 가져오기
            if(bIter==True):
                url_followlist = "https://api.twitch.tv/helix/users/follows?from_id=" + str(user_id) + "&first=100&after=" + str(cursor)
                response_followlist = requests.get(url_followlist, headers = header)
                res_json = response_followlist.json()

        following_list = ",".join(li_following)

        df_user_following.loc[len(df_user_following)] = {"id" : user_id, "following_list" : following_list}
    
        time.sleep(1)
        
    except Exception as e:
        print(e)
        time.sleep(10)
        pass

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [187]:
df_user_following.isnull().sum()

id                 0
following_list    12
dtype: int64

In [191]:
tmp = df_user_following.dropna()

In [192]:
# tmp.to_csv("data/user_following_list.csv", index = False)

In [ ]:
df_user_following = pd.read_csv("data/user_following_list.csv")

### Streamer list

In [17]:
li_folowing_list = df_user_following.following_list.tolist()

In [33]:
li_streamer_list = []
for element in tqdm_notebook(li_folowing_list):
    try :
        li_streamer_list += str(element).split(",")
        li_streamer_list = list(set(li_streamer_list))
    except Exception:
        print(element)

HBox(children=(IntProgress(value=0, max=21815), HTML(value='')))

In [162]:
with open("data/streamer_list.p", "rb") as f:
    li_streamer_list = pickle.load(f)

In [167]:
df_stream_info = pd.DataFrame(columns = ["id", "nickname", "name"])

In [168]:
for stream_id in tqdm_notebook(li_streamer_list):
    try :
        url_channel_info = "https://api.twitch.tv/kraken/channels/" + str(stream_id)
        response_channel_info = requests.get(url_channel_info, headers = header)
        res_json = response_channel_info.json()
        
        if res_json["error"] == 'Unprocessable Entity':
            unentity_list.append(stream_id)
            continue
        user_id = stream_id
        nickname = res_json["display_name"]
        name = res_json["name"]

        df_stream_info.loc[len(df_stream_info)] = {"id" : user_id, "nickname" : nickname, "name" : name}
        time.sleep(1)
    except Exception as e:
        print("wait", e)
        time.sleep(10)

HBox(children=(IntProgress(value=0, max=2675), HTML(value='')))

wait 'display_name'
wait HTTPSConnectionPool(host='api.twitch.tv', port=443): Max retries exceeded with url: /kraken/channels/143546995 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001924D478B00>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다',))
wait HTTPSConnectionPool(host='api.twitch.tv', port=443): Max retries exceeded with url: /kraken/channels/117037009 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001924B847668>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다',))
wait 'display_name'
wait HTTPSConnectionPool(host='api.twitch.tv', port=443): Max retries exceeded with url: /kraken/channels/78711184 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001924B8477B8>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 